In [ ]:
!git clone https://github.com/kk-digital/kcg-ml-sd1p4
!cd kcg-ml-sd1p4 && git pull

In [117]:
import torch
from PIL import ImageDraw, Image
from typing import List, Tuple

Point = Tuple[int, int]
Points = List[Point]

PRESERVE = 1
MODIFY = 0

In [118]:
def create_mask_from_points(points: Points, image_width: int, image_height: int, float_size=32):
    d_type = torch.float32
    if float_size == 64:
        d_type = torch.float64  
    elif float_size == 16:
        d_type = torch.float16

    mask = torch.zeros((image_height, image_width), dtype=d_type)
    mask = torch.fill(mask, PRESERVE)
    
    # mark the points in the mask with 1
    for point in points:
        x, y = point
        mask[x][y] = MODIFY

    return mask

In [119]:
def generate_points_within_polygon(vertices: Points):
    # Find the minimum and maximum x and y values
    min_x = min(vertices, key=lambda p: p[0])[0]
    max_x = max(vertices, key=lambda p: p[0])[0]
    min_y = min(vertices, key=lambda p: p[1])[1]
    max_y = max(vertices, key=lambda p: p[1])[1]

    # Generate random points within the bounding box
    points: Points = []
    for y in range(min_y, max_y + 1):
        intersections = []
        for i in range(len(vertices)):
            x1, y1 = vertices[i]
            x2, y2 = vertices[(i + 1) % len(vertices)]

            # Check if the point is within the polygon
            if (y1 <= y < y2) or y2 <= y < y1:
                intersections.append(int(x1 + (y - y1) * (x2 - x1) / (y2 - y1)))

        # Sort the intersections from smallest to largest
        intersections.sort()
        for i in range(0, len(intersections), 2):
            for x in range(intersections[i], intersections[i + 1] + 1):
                points.append((x, y))

    return points

In [120]:
d = 10
vertices = [(0, 0), (0, 5), (5, 0)]
points = generate_points_within_polygon(vertices)
mask = create_mask_from_points(points, d, d)
print(mask)

tensor([[0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])


In [121]:
def generate_points_within_circle(center: Point, radius: int):
    points: Points = []
    for x in range(center[0] - radius, center[0] + radius + 1):
        for y in range(center[1] - radius, center[1] + radius + 1):
            if (x - center[0]) ** 2 + (y - center[1]) ** 2 <= radius ** 2:
                points.append((x, y))

    return points

In [122]:
d = 10
center = (5, 5)
radius = 3
points = generate_points_within_circle(center, radius)
mask = create_mask_from_points(points, d, d)
print(mask)

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 0., 1., 1., 1., 1.],
        [1., 1., 1., 0., 0., 0., 0., 0., 1., 1.],
        [1., 1., 1., 0., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 0., 0., 0., 1.],
        [1., 1., 1., 0., 0., 0., 0., 0., 1., 1.],
        [1., 1., 1., 0., 0., 0., 0., 0., 1., 1.],
        [1., 1., 1., 1., 1., 0., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])


In [123]:
from PIL import Image
from typing import Union

def add_pixels_to_image(image: Union[str, Image.Image], top_pixels: int, right_pixels: int, bottom_pixels: int, left_pixels: int):
    if isinstance(image, str):
        image = Image.open(image)

    # Get the current image size
    width, height = image.size

    # Calculate the new image size
    new_width = width + left_pixels + right_pixels
    new_height = height + top_pixels + bottom_pixels

    # Create a new blank image with the desired size
    new_image = Image.new(image.mode, (new_width, new_height))

    # Paste the original image into the new image with the offsets
    new_image.paste(image, (left_pixels, top_pixels))

    return new_image

In [124]:
image_path = '../monkey.jpg'
top_pixels = bottom_pixels = left_pixels = right_pixels = 100

new_image = add_pixels_to_image(image_path, top_pixels, bottom_pixels, left_pixels, right_pixels)
new_image.save('monkey_with_border.jpg')


In [125]:
import numpy as np

def generate_inpaint_mask(width: int, height: int, top_pixels: int, right_pixels: int, bottom_pixels: int, left_pixels: int):
    # Create a blank mask of the same size as the image
    new_width = width + left_pixels + right_pixels
    new_height = height + top_pixels + bottom_pixels
    
    mask = np.full((new_height, new_width), PRESERVE, dtype=np.uint8)

    # Fill in the mask with MODIFY values where the image was padded
    mask[:top_pixels, :] = MODIFY
    mask[-bottom_pixels:, :] = MODIFY
    mask[:, :left_pixels] = MODIFY
    mask[:, -right_pixels:] = MODIFY

    return torch.from_numpy(mask)

In [126]:
width = height = 10
top_pixels = bottom_pixels = left_pixels = right_pixels = 2
mask = generate_inpaint_mask(width, height, top_pixels, right_pixels, bottom_pixels, left_pixels)
print(mask)

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=torch.uint8)


In [148]:
from typing import List, Optional
import torch

def merge_masks(masks: List[torch.Tensor]) -> torch.Tensor:
    if not masks:
        return torch.empty(0)

    # Get the shape of the first mask
    mask_shape = masks[0].shape

    # Create a merged mask tensor
    merged_mask = torch.full_like(masks[0], PRESERVE, dtype=torch.float32)

    for mask in masks:
        # Ensure that the shapes of all masks match
        if mask.shape != mask_shape:
            raise ValueError("All masks should have the same shape.")

        # Merge the current mask with the existing merged mask
        merged_mask = torch.logical_and(merged_mask, mask)

    # Convert the merged mask to a float tensor
    merged_mask = merged_mask.type(torch.float32)

    return merged_mask

In [150]:
width = height = 14

points1 = [(0, 0), (0, 5), (5, 0)]
points2 = [(5, 5), (0, 5), (5, 0)]

polygon1 = generate_points_within_polygon(points1)
polygon2 = generate_points_within_polygon(points2)

mask1 = create_mask_from_points(polygon1, width, height)
mask2 = create_mask_from_points(polygon2, width, height)

mask = merge_masks([mask1, mask2])
print(mask)


tensor([[0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])


In [129]:
def invert_mask(mask: torch.Tensor):
    return torch.logical_not(mask).int()

In [154]:
width = height = 14

# 3 points polygon
points1 = [(0, 0), (0, 5), (5, 0)]
points2 = [(5, 5), (0, 5), (5, 0)]

polygon1 = generate_points_within_polygon(points1)
polygon2 = generate_points_within_polygon(points2)

mask1 = create_mask_from_points(polygon1, width, height)
mask2 = create_mask_from_points(polygon2, width, height)

mask = merge_masks([mask1, mask2])
mask = invert_mask(mask)
print(mask)


tensor([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=torch.int32)


In [ ]:
from in_paint import InPaint

image_path = '../monkey.jpg'
image = Image.open(image_path)

width, height = image.size

top_pixels = bottom_pixels = left_pixels = right_pixels = 100
mask = generate_inpaint_mask(width, height, top_pixels, right_pixels, bottom_pixels, left_pixels)
new_image = add_pixels_to_image(image, top_pixels, right_pixels, bottom_pixels, left_pixels)


new_image_path = 'monkey_with_border.jpg'
new_image.save(new_image_path)

inpaint = InPaint(checkpoint_path='../input/model/sd-v1-4.ckpt')
inpaint.initialize_script()

strength = 0.5

# In-paint the image
images = inpaint.repaint_image(orig_img=new_image_path,
                        strength=strength,
                        batch_size=1,
                        prompt="a monkey",
)

def save_images(images: torch.Tensor, dest_path: str, img_format: str = 'jpeg'):
    """
    ### Save a images

    :param images: is the tensor with images of shape `[batch_size, channels, height, width]`
    :param dest_path: is the folder to save images in
    :param img_format: is the image format
    """

    # Map images to `[0, 1]` space and clip
    images = torch.clamp((images + 1.0) / 2.0, min=0.0, max=1.0)
    # Transpose to `[batch_size, height, width, channels]` and convert to numpy
    images = images.cpu()
    images = images.permute(0, 2, 3, 1)
    images = images.float().numpy()

    # Save images
    for i, img in enumerate(images):
        img = Image.fromarray((255. * img).astype(np.uint8))
        img.save(dest_path, format=img_format)

# Save the image
save_images(images, 'monkey_inpaint.jpg')
